In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib auto

cv2.__version__

Using matplotlib backend: MacOSX


'3.1.0'

In [4]:
idxList = []
def onclick(event):
    print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          (event.button, event.x, event.y, event.xdata, event.ydata))
    global idxList
    idxList.append([event.xdata, event.ydata])

def getPoint(x,y,img):
    temp = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return temp[x,y]

def getLineSegment(x1,x2,y,img):
    temp = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return temp[x1:x2,y]

def colorFromIdx(idx, frame):
    x,y = idx 
    return frame[int(y), int(x), :]

def plotMask(idx, frame):
    idColor = colorFromIdx(idx, frame)
    offset = 20
    mask = np.zeros(frame.shape)
    for ii,color in enumerate(idColor):
        mask[:,:,ii] = (frame[:,:,ii] > color-offset) & (frame[:,:,ii]<color+offset)

    plt.figure()
    plt.imshow(np.sum(mask,axis=2))
    plt.title(str(idx))

def plotRefPoint(idx, ax):
    x,y = idx 
    ax.scatter(int(x), int(y))
    
def getMask(idColor, frame):
    offset = 20
    mask = np.zeros(frame.shape)
    for ii,color in enumerate(idColor):
        mask[:,:,ii] = (frame[:,:,ii] > color-offset) & (frame[:,:,ii]<color+offset) 
    mask = np.sum(mask, axis=2) ==3
    return mask

def getBlob(mask):
    blob = cv2.dilate(mask.astype(np.uint8), kernel)
    return blob

def getCentroid(blob):
    temp = blob.copy()
    temp[temp==0] = 255
    temp[temp==1] = 0
    keypoints = detector.detect(temp)
    return keypoints

In [34]:
cap = cv2.VideoCapture('test.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES,0)
fs = cap.get(cv2.CAP_PROP_FPS)

ret,frame = cap.read()

# setup initial location of window
r,h,c,w = 330,50,360,50  # simply hardcoded the values
track_window = (c,r,w,h)

# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret ,frame = cap.read()

    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw it on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv2.imshow('img2',img2)

        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break

    else:
        break

cv2.destroyAllWindows()
cap.release()

True

array([[   0.        ],
       [   0.        ],
       [   6.71052647],
       [  13.42105293],
       [   0.        ],
       [   0.        ],
       [ 150.98684692],
       [   0.        ],
       [  30.19736862],
       [ 110.72368622],
       [ 134.21052551],
       [ 120.78947449],
       [ 255.        ],
       [  53.68421173],
       [  57.03947449],
       [  97.30263519],
       [ 161.05264282],
       [  67.10526276],
       [  33.55263138],
       [   3.35526323],
       [   6.71052647],
       [  46.97368622],
       [   0.        ],
       [   0.        ],
       [  13.42105293],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.        ],
       [   0.   

In [9]:
frame

array([[[ 26,  26,  31],
        [ 27,  27,  32],
        [ 27,  27,  32],
        ..., 
        [241, 250, 242],
        [235, 244, 236],
        [241, 250, 242]],

       [[ 26,  26,  31],
        [ 27,  27,  32],
        [ 27,  27,  32],
        ..., 
        [247, 255, 248],
        [246, 255, 247],
        [240, 249, 241]],

       [[ 26,  26,  31],
        [ 27,  27,  32],
        [ 27,  27,  32],
        ..., 
        [246, 255, 247],
        [247, 255, 248],
        [246, 255, 247]],

       ..., 
       [[148, 151, 181],
        [138, 141, 171],
        [189, 200, 214],
        ..., 
        [ 19,  29,  39],
        [ 19,  29,  39],
        [ 19,  29,  39]],

       [[148, 179, 188],
        [202, 233, 242],
        [209, 250, 255],
        ..., 
        [ 19,  29,  39],
        [ 19,  29,  39],
        [ 19,  29,  39]],

       [[219, 250, 255],
        [218, 249, 255],
        [181, 222, 235],
        ..., 
        [ 19,  29,  39],
        [ 19,  29,  39],
        [ 19,  29,

In [28]:
frameOffset = 300
bufLen = 5

# get these with mouse clicks later
x1 = 150
x2 = 470
y1 = 290
y2 = 450

# grab and display a frame during run
cap = cv2.VideoCapture('test.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES,frameOffset)
fs = cap.get(cv2.CAP_PROP_FPS)

ret,frame = cap.read()
frame = frame[x1:x2, y1:y2, :]
figRef = plt.figure()
axRef = figRef.add_subplot(111)
axRef.imshow(frame)

# mouse click on columns to histogram
cid = figRef.canvas.mpl_connect('button_press_event', onclick);

# Set up the SimpleBlobdetector with default parameters.
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 0;
params.maxThreshold = 256;

# Filter by Area.
params.filterByArea = True
params.minArea = 5

# Filter by Circularity
params.filterByCircularity = False
params.minCircularity = 0.1

# Filter by Convexity
params.filterByConvexity = False
params.minConvexity = 0.5

# Filter by Inertia
params.filterByInertia = False
params.minInertiaRatio = 0.5

detector = cv2.SimpleBlobDetector_create(params)


True

button=1, x=289, y=180, xdata=47.000000, ydata=209.500000
button=1, x=288, y=182, xdata=46.166667, ydata=207.833333
button=1, x=288, y=182, xdata=46.166667, ydata=207.833333
button=1, x=288, y=188, xdata=46.166667, ydata=202.833333


In [29]:
figRef.canvas.mpl_disconnect(cid);
    
for idx in idxList:    
    plotMask(idx,frame)
    plotRefPoint(idx,axRef)
    


In [39]:
DISP = False
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (30,30))
frameSkip = 0

# add user input to choose the best starting point
idx = idxList[1]
x,y = idx
idColor = colorFromIdx(idx, frame)
xBuff = [x]
yBuff = [y]

cap = cv2.VideoCapture('test.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES,frameSkip)
ret,frame = cap.read()
count = 0

while ret:
    count = count + 1
    frame =  frame[x1:x2, y1:y2, :]
    mask = getMask(idColor, frame)
    blob = getBlob(mask)
    keypoints = getCentroid(blob)
    
    # if we find any blobs
    if keypoints:
        
        # assume one and use that as the centroid
        x,y = keypoints[0].pt
        
        # if more than one find the one closest to the last one
        minVal = float('inf')
        for kp in keypoints:
            x,y = kp.pt
            dist = np.power((x-xBuff[-1]),2)+np.power((y-yBuff[-1]),2)
            if dist < minVal:
                minVal = dist
                xOut = x
                yout = y
                    
                
    else:
        
        # if no blob found then use the old value
        # mostly we get here because the blob touches the edge of the image
        x = xBuff[-1]
        y = yBuff[-1]
        #input('')
        
    xBuff.append(x)
    yBuff.append(y)
    
    
    if DISP:
        new = cv2.circle(cv2.bitwise_and(frame, frame, mask=blob), (int(x),int(y)), 5, 255, -1)
        cv2.imshow('frame',new)
#         if cv2.waitKey(20) & 0xFF == ord('q'):
#             break
        k = cv2.waitKey(0)
        if k == 27:         # wait for ESC key to exit
            cv2.destroyAllWindows()
            break
        elif k == ord('n'): # wait for 's' for next frame
            t = 1
            
    ret,frame = cap.read()


cap.release()



True

KeyboardInterrupt: 

In [23]:
cv2.destroyAllWindows()


In [24]:
np.max(blob)

1

In [25]:
plt.imshow(result)

NameError: name 'result' is not defined

In [40]:
cv2.destroyAllWindows()

# user input to figure out true offset
frameOffset = 500

# low pass filter to smooth
N=10
fc = fs/40.

import scipy.signal as sp

# use firwin with arbitrary Fs
h=sp.firwin( numtaps=N, cutoff=fc, nyq=fs/2)
filtY=sp.lfilter( h, 1.0, yBuff) # 'x' is the time-series data you are filtering


# count number of positive inflextions points (first derivative zero / second neg)
dy = np.diff(filtY)
dy[dy>0] = 1
dy[dy<0] = -1
ddy = np.diff(dy)
ddy[ddy>0] = 0
totalSteps = np.count_nonzero(ddy[frameOffset:25700])

print('Total Steps = %d'%totalSteps)

Total Steps = 97


In [41]:
plt.figure()
plt.plot(np.power(yBuff,2)+np.power(xBuff,2))
plt.figure()
plt.plot(yBuff)
